In [1]:
import pandas as pd
import mysql.connector as msql
from mysql.connector import Error
import warnings
warnings.filterwarnings("ignore")

Para demostrar lo aprendido en la clase, en esta libreta se realizara la carga de informacion desde un archivo csv a la base de datos de tipo MySql.

Primeramente, mi ambiente para desarrollar es WSL, el subsistema de linux usado desde Windows, para realizar la conexion con el servidor local fue necesario instalar MySql desde la terminal de ubuntu, para ello segui esta [guia: "How to install MySQL on WSL 2](https://pen-y-fan.github.io/2021/08/08/How-to-install-MySQL-on-WSL-2-Ubuntu/), es muy directa y saca del apuro, sobre todo lo relacionado con la primera corrida y el establecer la contraseña para el usuario base root.

Ademas antes de empezar hay que inicializar el servidor, en nuestro caso con el siguiente comando: 
**mysql -u root -h 127.0.0.1 -p -P 3306**

# Coordenadas
Una vez que ya tenemos el servidor corriendo, procedemos a obtener los datos desde github

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/mcd-unison/ing-caract/main/ejemplos/mapas/python/municipios_latlon.csv')
df.head()

,MUNICIPIO,LATITUD,LONGITUD
0,ACONCHI,29.825420,-110.20840
1,AGUA PRIETA,31.330048,-109.54904
2,ALAMOS,27.026820,-108.92560
3,ALTAR,30.715070,-111.82135
4,ARIVECHI,29.078530,-109.19583


Primeramente establecemos la conexion, y procedemos a crear la base de datos en la que guardaremos las tablas necesaria para la creacion de nuestros mapas

In [3]:
try:
    conn = msql.connect(host='localhost', user='root', password= 'root' ) #Establecer conexion
    if conn.is_connected():
        cursor = conn.cursor()#Crear cursor
        cursor.execute("CREATE DATABASE sonora") #Ejecutar instruccion
        print("La base de datos sonora fue creada")
        #Normalmente aqui se cierra el conector pero lo usaremos mas adelante en la libreta
except Error as e:
    print("Error al conectarse: ", e)

Error al conectarse:  1007 (HY000): Can't create database 'sonora'; database exists


In [4]:
df.dtypes

MUNICIPIO     object
LATITUD      float64
LONGITUD     float64
dtype: object

Con la base de datos ya creada procedemos a crear la tabla que contendra las coordenadas de los municipios de Sonora

In [5]:
try:
    conn = msql.connect(host='localhost',
                           database='sonora', user='root',
                           password='root')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("Estas coenctado a: ", record)
        cursor.execute('DROP TABLE IF EXISTS coordenada;')#Si existe una tabla con este nombre la borramos
        print('Creado tabla coordenada...')#Creacion de tabla con tipos y restricciones de datos
        cursor.execute(f"""CREATE TABLE coordenada (municipio CHAR(100) 
                        NOT NULL, latitud FLOAT(9,6) NOT NULL, 
                        longitud FLOAT(9,6) NOT NULL)""")
        print("La tabla coordenada fue creada...")
        for i,row in df.iterrows():#Insercion de valores
            sql = "INSERT INTO sonora.coordenada VALUES (%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            #La conexion no confirma los cambios en automatico por default, por lo que debemos correr el siguiente comando
            conn.commit()
        print('Operacion exitosa')
except Error as e:
    print("Error al conectarse: ", e)

Estas coenctado a:  ('sonora',)
Creado tabla coordenada...
La tabla coordenada fue creada...
Operacion exitosa


In [6]:
# Corremos un query
sql = "SELECT * FROM coordenada"
cursor.execute(sql)
# Jalar todos los resultados
result = cursor.fetchall()
for i in result:
    print(i)

('ACONCHI', 29.82542, -110.208397)
('AGUA PRIETA', 31.330048, -109.549042)
('ALAMOS', 27.026819, -108.925598)
('ALTAR', 30.715071, -111.82135)
('ARIVECHI', 29.078529, -109.195831)
('ARIZPE', 30.40764, -110.164749)
('ATIL', 30.884211, -111.582779)
('BACADEHUACHI', 29.81189, -109.140793)
('BACANORA', 28.98597, -109.400398)
('BACERAC', 30.50449, -108.924011)
('BACOACHI', 30.633801, -109.968391)
('BACUM', 27.55286, -110.0821)
('BANAMICHI', 30.00947, -110.213371)
('BAVIACORA', 29.71299, -110.161568)
('BAVISPE', 30.480261, -108.940491)
('BENITO JUAREZ', 27.116699, -109.833298)
('BENJAMIN HILL', 30.16931, -111.115051)
('CABORCA', 30.714491, -112.161087)
('CAJEME', 27.4974, -109.91124)
('CANANEA', 30.989941, -110.289169)
('CARBO', 29.68442, -110.956177)
('CUCURPE', 30.33173, -110.701332)
('CUMPAS', 29.99502, -109.780388)
('DIVISADEROS', 29.614441, -109.470093)
('EMPALME', 27.966631, -110.813652)
('ETCHOJOA', 26.91144, -109.626701)
('FRONTERAS', 30.89702, -109.558647)
('GENERAL PLUTARCO ELIAS C

# Poblacion
Realizamos el proceso anterior de manera analogo para los datos de la poblacion

In [7]:
df = pd.read_csv('https://raw.githubusercontent.com/mcd-unison/ing-caract/main/ejemplos/mapas/python/POBLACION_SONORA2020.csv')
df.head()

,JURISDICCION SANITARIA,COORDINACION MEDICA LOCAL,MUNICIPIO,POBLACION TOTAL
0,JURISDICCION SANITARIA I - HERMOSILLO,CML 01 HERMOSILLO,CARBO,6261
1,JURISDICCION SANITARIA I - HERMOSILLO,CML 01 HERMOSILLO,LA COLORADA,1675
2,JURISDICCION SANITARIA I - HERMOSILLO,CML 01 HERMOSILLO,HERMOSILLO,914163
3,JURISDICCION SANITARIA I - HERMOSILLO,CML 01 HERMOSILLO,MAZATAN,1403
4,JURISDICCION SANITARIA I - HERMOSILLO,CML 01 HERMOSILLO,ONAVAS,421


In [8]:
df.dtypes

JURISDICCION SANITARIA       object
COORDINACION MEDICA LOCAL    object
MUNICIPIO                    object
POBLACION TOTAL               int64
dtype: object

In [9]:
try:#Mismo proceso para creacion de tablas que se realizo antes
    conn = msql.connect(host='localhost',
                           database='sonora', user='root',
                           password='root')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("Estas coenctado a: ", record)
        cursor.execute('DROP TABLE IF EXISTS poblacion;')
        print('Creado tabla poblacion...')
        cursor.execute(f"""CREATE TABLE poblacion (
                        jurisdiccion_sanitaria CHAR(100) NOT NULL,
                        coord_medica_local CHAR(100) NOT NULL, 
                        municipio CHAR(100) NOT NULL, 
                        poblacion INT NOT NULL)""")
        print("La tabla coordenada fue creada...")
        for i,row in df.iterrows():
            sql = "INSERT INTO sonora.poblacion VALUES (%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            # the connection is not autocommitted by default, so we must commit to save our changes
            conn.commit()
        print('Operacion exitosa')
except Error as e:
    print("Error al conectarse: ", e)

Estas coenctado a:  ('sonora',)
Creado tabla poblacion...
La tabla coordenada fue creada...
Operacion exitosa


In [10]:
# Execute query
sql = "SELECT * FROM poblacion"
cursor.execute(sql)
# Fetch all the records
result = cursor.fetchall()
for i in result:
    print(i)

('JURISDICCION SANITARIA I - HERMOSILLO', 'CML 01 HERMOSILLO', 'CARBO', 6261)
('JURISDICCION SANITARIA I - HERMOSILLO', 'CML 01 HERMOSILLO', 'LA COLORADA', 1675)
('JURISDICCION SANITARIA I - HERMOSILLO', 'CML 01 HERMOSILLO', 'HERMOSILLO', 914163)
('JURISDICCION SANITARIA I - HERMOSILLO', 'CML 01 HERMOSILLO', 'MAZATAN', 1403)
('JURISDICCION SANITARIA I - HERMOSILLO', 'CML 01 HERMOSILLO', 'ONAVAS', 421)
('JURISDICCION SANITARIA I - HERMOSILLO', 'CML 01 HERMOSILLO', 'SAN JAVIER', 745)
('JURISDICCION SANITARIA I - HERMOSILLO', 'CML 01 HERMOSILLO', 'SAN MIGUEL DE HORCASITAS', 10939)
('JURISDICCION SANITARIA I - HERMOSILLO', 'CML 01 HERMOSILLO', 'SAN PEDRO DE LA CUEVA', 1815)
('JURISDICCION SANITARIA I - HERMOSILLO', 'CML 01 HERMOSILLO', 'SOYOPA', 1373)
('JURISDICCION SANITARIA I - HERMOSILLO', 'CML 01 HERMOSILLO', 'SUAQUI GRANDE', 1171)
('JURISDICCION SANITARIA I - HERMOSILLO', 'CML 01 HERMOSILLO', 'VILLA PESQUEIRA', 1242)
('JURISDICCION SANITARIA I - HERMOSILLO', 'CML 02 URES', 'ACONCHI', 

In [11]:
# Close the connection
if (conn.is_connected()):
    cursor.close()
    conn.close()
    print("MySQL cerrada")

MySQL cerrada


# Leyendo lo que escribimos
Creamos un conector que sera usado por pandas

In [12]:
try:
    mydb = msql.connect(host="localhost", database = 'sonora',user="root", passwd="root",use_pure=True)
    query_pob = "Select * from poblacion;"
    municipios_poblacion  = pd.read_sql(query_pob,mydb)
    query_mun = "Select * from coordenada;"
    municipios = pd.read_sql(query_mun,mydb)
    mydb.close() #close the connection
except Exception as e:
    mydb.close()
    print(str(e))

In [13]:
municipios = municipios.merge(municipios_poblacion, on="municipio") 

municipios.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72 entries, 0 to 71
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   municipio               72 non-null     object 
 1   latitud                 72 non-null     float64
 2   longitud                72 non-null     float64
 3   jurisdiccion_sanitaria  72 non-null     object 
 4   coord_medica_local      72 non-null     object 
 5   poblacion               72 non-null     int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 3.9+ KB


In [14]:
try:
    mydb = msql.connect(host="localhost", database = 'sonora',user="root", passwd="root",use_pure=True)
    query = f"""SELECT c.municipio, c.latitud,c.longitud,
                        p.jurisdiccion_sanitaria,p.coord_medica_local, p.poblacion 
                        FROM coordenada c INNER JOIN poblacion p ON p.municipio = c.municipio;"""
    municipios_join  = pd.read_sql(query,mydb)
    mydb.close() #close the connection
except Exception as e:
    mydb.close()
    print(str(e))

In [15]:
municipios_join.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   municipio               72 non-null     object 
 1   latitud                 72 non-null     float64
 2   longitud                72 non-null     float64
 3   jurisdiccion_sanitaria  72 non-null     object 
 4   coord_medica_local      72 non-null     object 
 5   poblacion               72 non-null     int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 3.5+ KB


In [16]:
municipios_join = municipios_join.sort_values(by=['municipio']).reset_index(drop=True)

Despues de una ordenada, podemos observar que los resulados son equivalentes

In [17]:
municipios_join.head()

,municipio,latitud,longitud,jurisdiccion_sanitaria,coord_medica_local,poblacion
0,ACONCHI,29.825420,-110.208397,JURISDICCION SANITARIA I - HERMOSILLO,CML 02 URES,2928
1,AGUA PRIETA,31.330048,-109.549042,JURISDICCION SANITARIA III - SANTA ANA,CML 12 AGUA PRIETA,92364
2,ALAMOS,27.026819,-108.925598,JURISDICCION SANITARIA V - NAVOJOA,CML 18 ALAMOS,28461
3,ALTAR,30.715071,-111.821350,JURISDICCION SANITARIA II - CABORCA,CML 07 CABORCA,10232
4,ARIVECHI,29.078529,-109.195831,JURISDICCION SANITARIA I - HERMOSILLO,CML 03 SAHUARIPA,1304


In [18]:
municipios.head()

,municipio,latitud,longitud,jurisdiccion_sanitaria,coord_medica_local,poblacion
0,ACONCHI,29.825420,-110.208397,JURISDICCION SANITARIA I - HERMOSILLO,CML 02 URES,2928
1,AGUA PRIETA,31.330048,-109.549042,JURISDICCION SANITARIA III - SANTA ANA,CML 12 AGUA PRIETA,92364
2,ALAMOS,27.026819,-108.925598,JURISDICCION SANITARIA V - NAVOJOA,CML 18 ALAMOS,28461
3,ALTAR,30.715071,-111.821350,JURISDICCION SANITARIA II - CABORCA,CML 07 CABORCA,10232
4,ARIVECHI,29.078529,-109.195831,JURISDICCION SANITARIA I - HERMOSILLO,CML 03 SAHUARIPA,1304


# Haciendo mapas con folium

In [19]:
#!conda install folium -c conda-forge -y

In [20]:
# Creando un mapa sólo para ver:

import folium

m = folium.Map(
    location=(29.66667, -110.5),
    tiles='OpenStreetMap',
    zoom_start=7,
    min_zoom=4,
    max_zoom=12,
    width='80%',
    height='80'
)
m

In [21]:
m = folium.Map(
    location=(29.66667, -110.5),
    tiles='OpenStreetMap',
    zoom_start=7,
    min_zoom=4,
    max_zoom=12,
)


for row in municipios.itertuples():
    folium.Marker(
        location=[row.latitud, row.longitud],
        tooltip=f"{row.municipio} Población: {row.poblacion} habitantes",
    ).add_to(m)
m

In [22]:
from folium.plugins import MarkerCluster

m = folium.Map(
    location=(29.66667, -110.5),
    tiles='OpenStreetMap',
    zoom_start=7,
    min_zoom=4,
    max_zoom=12,
)

grupo = MarkerCluster().add_to(m)

for row in municipios.itertuples():
    folium.Marker(
        location=[row.latitud, row.longitud],
        tooltip=f"{row.municipio} Población: {row.poblacion} habitantes",
    ).add_to(grupo)
m

# Creating a view

In [23]:
try:
    conn = msql.connect(host='localhost',
                           database='sonora', user='root',
                           password='root')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("Estas coenctado a: ", record)
        cursor.execute('DROP VIEW IF EXISTS vw_mapa;')
        print('Creado tabla poblacion...')
        cursor.execute(f"""CREATE VIEW vw_mapa AS
                               SELECT c.municipio, c.latitud,c.longitud,
                                      p.jurisdiccion_sanitaria,p.coord_medica_local, p.poblacion 
                               FROM coordenada c INNER JOIN poblacion p ON p.municipio = c.municipio
                               ORDER BY municipio
                               ;
        """)
        print("La vista vw_mapa fue creada...")
        print('Operacion exitosa')
except Error as e:
    print("Error al conectarse: ", e)

Estas coenctado a:  ('sonora',)
Creado tabla poblacion...
La vista vw_mapa fue creada...
Operacion exitosa


In [24]:
try:
    query_pob = "Select * from vw_mapa;"
    municipios  = pd.read_sql(query_pob,conn)
except Exception as e:
    print(str(e))

In [25]:
municipios.head()

,municipio,latitud,longitud,jurisdiccion_sanitaria,coord_medica_local,poblacion
0,ACONCHI,29.825420,-110.208397,JURISDICCION SANITARIA I - HERMOSILLO,CML 02 URES,2928
1,AGUA PRIETA,31.330048,-109.549042,JURISDICCION SANITARIA III - SANTA ANA,CML 12 AGUA PRIETA,92364
2,ALAMOS,27.026819,-108.925598,JURISDICCION SANITARIA V - NAVOJOA,CML 18 ALAMOS,28461
3,ALTAR,30.715071,-111.821350,JURISDICCION SANITARIA II - CABORCA,CML 07 CABORCA,10232
4,ARIVECHI,29.078529,-109.195831,JURISDICCION SANITARIA I - HERMOSILLO,CML 03 SAHUARIPA,1304


In [26]:
# Close the connection
if (conn.is_connected()):
    cursor.close()
    conn.close()
    print("MySQL cerrada")

MySQL cerrada


# Creating a function

Por fines didacticos esto lo haremos de manera manual mediante la linea de comandos. 
```SQL
DROP FUNCTION IF EXISTS fn_get_poblacion_jurisdiccion
DELIMITER //
CREATE FUNCTION sonora.fn_get_poblacion_jurisdiccion(jurisdiccion CHAR(200)) 
RETURNS INT 
DETERMINISTIC
BEGIN
   declare jur_poblacion INT;
   SELECT SUM(poblacion) INTO jur_poblacion
   FROM vw_mapa WHERE jurisdiccion_sanitaria = jurisdiccion;
   RETURN jur_poblacion;
END //
DELIMITER ;
```

In [27]:
try:
    conn = msql.connect(host='localhost',
                           database='sonora', user='root',
                           password='root')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("Estas coenctado a: ", record)
        query_pob_hermosillo = "Select fn_get_poblacion_jurisdiccion('JURISDICCION SANITARIA I - HERMOSILLO');"
        poblacion_hermosillo  = pd.read_sql(query_pob_hermosillo,conn)
        query_pob_santaana = "Select fn_get_poblacion_jurisdiccion('JURISDICCION SANITARIA III - SANTA ANA');"
        poblacion_santaana  = pd.read_sql(query_pob_santaana,conn)
        print('Operacion exitosa')
except Error as e:
    print("Error al conectarse: ", e)

Estas coenctado a:  ('sonora',)
Operacion exitosa


In [28]:
poblacion_santaana

,fn_get_poblacion_jurisdiccion('JURISDICCION SANITARIA III - SANTA ANA')
0,492356


In [29]:
poblacion_hermosillo

,fn_get_poblacion_jurisdiccion('JURISDICCION SANITARIA I - HERMOSILLO')
0,1012683


In [30]:
# Close the connection
if (conn.is_connected()):
    cursor.close()
    conn.close()
    print("MySQL cerrada")

MySQL cerrada


# Creating a procedure

```SQL
DELIMITER $$
CREATE PROCEDURE fn_get_poblacion_coord_medica(IN coord CHAR(200))
BEGIN
SELECT
   SUM(poblacion) AS poblacion_coord
   FROM vw_mapa WHERE coord_medica_local = coord;
END $$
DELIMITER ;
```

In [31]:
try:
    conn = msql.connect(host='localhost',
                           database='sonora', user='root',
                           password='root')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("Estas coenctado a: ", record)
        query_pob_caborca = "CALL fn_get_poblacion_coord_medica('CML 07 CABORCA');"
        poblacion_caborca  = pd.read_sql(query_pob_caborca,conn)
        print('Operacion exitosa')
except Error as e:
    print("Error al conectarse: ", e)

Estas coenctado a:  ('sonora',)
Operacion exitosa


In [32]:
poblacion_caborca

,SUM(poblacion)
0,122597.0


## Exportando la base de datos.

Por ultimo usamos los comando que se usarian en la consola del sistema para exportar la base de datos.

In [33]:
import os
os.system('mysql -u root -p%s sonora > sonora.sql' % 'root')

mysql: [Warning] Using a password on the command line interface can be insecure.


0